In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import numpy as np

In [18]:
# Завантаження датасету Fashion MNIST
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Нормалізація зображень до діапазону [0, 1]
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Зміна розмірів зображень до 32x32 (оскільки VGG16 очікує 32x32 зображення)
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)
train_images = tf.image.resize(train_images, [32, 32])
test_images = tf.image.resize(test_images, [32, 32])

# Перетворення міток у категоріальний формат
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Додавання каналу для RGB
train_images = np.repeat(train_images, 3, axis=-1)
test_images = np.repeat(test_images, 3, axis=-1)

In [19]:
# Завантаження VGG16 без верхніх шарів (повнозв'язних)
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Замороження шарів VGG16
base_model.trainable = False

# Створення власної моделі
model = models.Sequential()

# Додавання VGG16 як основи
model.add(base_model)

# Вирівнювання даних для повнозв'язних шарів
model.add(layers.Flatten())

# Повнозв'язний шар
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

# Вихідний шар
model.add(layers.Dense(10, activation='softmax'))

# Виведення архітектури моделі
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [20]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, batch_size=128, validation_split=0.2)

Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1720s 5s/step - accuracy: 0.6441 - loss: 1.0356 - val_accuracy: 0.8151 - val_loss: 0.5065
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1613s 4s/step - accuracy: 0.8160 - loss: 0.5152 - val_accuracy: 0.8337 - val_loss: 0.4477
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1568s 4s/step - accuracy: 0.8357 - loss: 0.4604 - val_accuracy: 0.8416 - val_loss: 0.4274
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1743s 5s/step - accuracy: 0.8430 - loss: 0.4302 - val_accuracy: 0.8469 - val_loss: 0.4074
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1715s 5s/step - accuracy: 0.8515 - loss: 0.4140 - val_accuracy: 0.8533 - val_loss: 0.3946


In [21]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Точність на тестових даних: {test_acc:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 350s 1s/step - accuracy: 0.8556 - loss: 0.3903
Точність на тестових даних: 0.8532


In [23]:
# Створення багатошарової мережі
mlp_model = models.Sequential()

# Додавання шару Flatten для вирівнювання даних
mlp_model.add(layers.Flatten(input_shape=(32, 32, 3)))

# Повнозв'язні шари
mlp_model.add(layers.Dense(128, activation='relu'))
mlp_model.add(layers.Dense(64, activation='relu'))

# Вихідний шар
mlp_model.add(layers.Dense(10, activation='softmax'))

# Компіляція та навчання
mlp_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

mlp_history = mlp_model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

# Оцінка на тестових даних
mlp_test_loss, mlp_test_acc = mlp_model.evaluate(test_images, test_labels)
print(f'Точність багатошарової мережі на тестових даних: {mlp_test_acc:.4f}')

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 89s 107ms/step - accuracy: 0.7544 - loss: 0.6730 - val_accuracy: 0.8466 - val_loss: 0.4364
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.8489 - loss: 0.4181 - val_accuracy: 0.8475 - val_loss: 0.4092
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8671 - loss: 0.3640 - val_accuracy: 0.8664 - val_loss: 0.3836
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.8751 - loss: 0.3452 - val_accuracy: 0.8687 - val_loss: 0.3649
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8764 - loss: 0.3337 - val_accuracy: 0.8752 - val_loss: 0.3454
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8718 - loss: 0.3577
Точність багатошарової мережі на тестових даних: 0.8683
